## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import json
from datetime import datetime
from slugify import slugify

## Download

Retrieve the page

In [4]:
url = "https://services.arcgis.com/yq3FgOI44hYHAFVZ/arcgis/rest/services/covid19_city_all/FeatureServer/2//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(
        county="San Mateo",
        area=item["attributes"]["place"],
        confirmed_cases=item["attributes"]["n"],
    )
    dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

In [10]:
df["confirmed_cases"] = df.confirmed_cases.fillna("<10")

Get timestamp

In [11]:
date_url = "https://services.arcgis.com/yq3FgOI44hYHAFVZ/arcgis/rest/services/covid19_city_all/FeatureServer/2?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [12]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [13]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [14]:
latest_date = pd.to_datetime(timestamp).date()

In [15]:
df["county_date"] = latest_date

## Vet

In [16]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("San Mateo County's scraper has more rows than before")

In [17]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("San Mateo County's scraper is missing rows")

AssertionError: San Mateo County's scraper is missing rows

## Export

Set date

In [28]:
tz = pytz.timezone("America/Los_Angeles")

In [29]:
today = datetime.now(tz).date()

In [30]:
slug = "san-mateo"

In [31]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [32]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [33]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [34]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [35]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)